# L1000 repurpoing 

In [1]:
import numpy as np
import scipy.stats as ss
import h5py
import csv

## Load functions

In [6]:
# Input rank of zscore profile, and positions of up and down regulated genes to calculate ES score

def calc_ES(zscorerank,qup,qdown):  
    n = len(zscorerank)
    nu = len(qup)
    nd = len(qdown)

    uptemp = np.array(sorted(zscorerank[qup]))
    downtemp = np.array(sorted(zscorerank[qdown]))
    upmax = max(np.array(range(1, nu+1))/nu - uptemp/n)
    upmin = min(np.array(range(0, nu))/nu - (uptemp - 1)/n)
    downmax = max(np.array(range(1, nd+1))/nd - downtemp/n)
    downmin = min(np.array(range(0, nd))/nd - (downtemp - 1)/n)

    upscore = (upmax if abs(upmax) > abs(upmin) else upmin)
    downscore = (downmax if abs(downmax) > abs(downmin) else downmin)
  
    ES= ((downscore - upscore)/2 if(upscore*downscore < 0) else 0)
    
    return ES



In [40]:
def read_csv(f):
    l=[]
    with open(f) as csv_file:
        f_reader=csv.reader(csv_file)
        for row in f_reader:
            l.append(row)
    return l

def read_tsv(f):
    l=[]
    with open(f) as csv_file:
        f_reader=csv.reader(csv_file,delimiter='\t')
        for row in f_reader:
            l.append(row)
    return l

## Import L1000 data

In [41]:
data_path="/path/to/L1000_files/"
L1000_ph1 = h5py.File(data_path+'Bayesian_GSE92742_Level5_COMPZ_n361481x978.h5', 'r')
L1000_ph2 = h5py.File(data_path+'Bayesian_GSE70138_Level5_COMPZ_n116218x978.h5', 'r')

ph1_siginfo= read_tsv(data_path+"GSE92742_Broad_LINCS_sig_info.txt")
ph2_siginfo= read_tsv(data_path+"GSE70138_Broad_LINCS_sig_info.txt")


In [47]:
ph1_sig_dict={}
for i in range(len(ph1_siginfo)):
    ph1_sig_dict[ph1_siginfo[i][0]]=ph1_siginfo[i]
    
ph2_sig_dict={}
for i in range(len(ph2_siginfo)):
    ph2_sig_dict[ph1_siginfo[i][0]]=ph2_siginfo[i]

## Import up/down regulated genes

In [51]:
gene_path="/path/to/gene_files/"
up_genes= read_csv(gene_path+"up_genes.csv")[0]
down_genes= read_csv(gene_path+"down_genes.csv")[0]

## Process data

In [92]:
ph1_sigs=[x.decode() for x in L1000_ph1['colid']]
ph2_sigs=[x.decode() for x in L1000_ph2['colid']]
lm_genes=[x.decode() for x in L1000_ph1['rowid']]

ph1_data=np.array(L1000_ph1['data'])
ph2_data=np.array(L1000_ph2['data'])

lm_dict={}
for i in range(len(lm_genes)):
    lm_dict[lm_genes[i]]=i

ph1_sig_dict={}
for i in range(len(ph1_siginfo)):
    ph1_sig_dict[ph1_siginfo[i][0]]=ph1_siginfo[i]
    
ph2_sig_dict={}
for i in range(len(ph2_siginfo)):
    ph2_sig_dict[ph2_siginfo[i][0]]=ph2_siginfo[i]


L1000_up_genes=list(set([g.strip() for g in up_genes]) & set(lm_genes))
L1000_down_genes=list(set([g.strip() for g in down_genes]) & set(lm_genes))

qup=[lm_dict[g] for g in L1000_up_genes]
qdown=[lm_dict[g] for g in L1000_down_genes]

## Calculate enrichment scrores for all L1000 profiles

In [65]:
ph1_ranked=[ss.rankdata(i) for i in ph1_data]
ph2_ranked=[ss.rankdata(i) for i in ph2_data]

In [81]:
ES_list1=[]
for i in range(len(ph1_sigs)):
    score= calc_ES(ph1_ranked[i],qup,qdown)
    ES_list1.append(score)

ES_list2=[]
for i in range(len(ph2_sigs)):
    score= calc_ES(ph2_ranked[i],qup,qdown)
    ES_list2.append(score)


## Top sigs

In [82]:
top_N=10

In [93]:
sort_index1 = np.argsort(ES_list1)
sort_index2 = np.argsort(ES_list2)

ph1_topsigs=[ph1_sigs[i] for i in sort_index1[-top_N:]]
ph2_topsigs=[ph2_sigs[i] for i in sort_index2[-top_N:]]

ph1_topinfo=[ph1_sig_dict[i] for i in ph1_topsigs]
ph2_topinfo=[ph2_sig_dict[i] for i in ph2_topsigs]
